# PCA

A análise de componentes será feita utilizando o método jackknife para encontrar o número ótimo de componentes do PCA. Depois é aplicado o método Kennard-Stone para dividir as amostras em 70-30. Plota gráficos de outliers (T² e Q).

# Imports


In [9]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
import plotly.express as px
import seaborn as sns

# Método Jackknife

In [10]:
def jackknife_pca(X, threshold=0.01):
    n_samples = X.shape[0]
    variances = []
    
    for i in range(n_samples):
        X_jack = np.delete(X, i, axis=0)
        pca = PCA().fit(X_jack)
        variances.append(pca.explained_variance_ratio_)
    
    variances = np.mean(variances, axis=0)
    
    for i in range(len(variances) - 1):
        if variances[i] - variances[i + 1] < threshold:
            return i + 1
    return len(variances)

# Método Kennard-Stone

In [11]:
def kennard_stone(X, train_size=0.7):
    n_samples = X.shape[0]
    n_train = int(n_samples * train_size)

    if n_train < 2:
        raise ValueError("O conjunto de treinamento precisa ter pelo menos duas amostras!")

    # Calcula as distâncias entre cada par de pontos
    distances = cdist(X, X, metric='euclidean')

    # Primeiro ponto: aquele mais distante da média
    mean_sample = np.mean(X, axis=0)
    first_sample = np.argmax(np.linalg.norm(X - mean_sample, axis=1))
    selected = [first_sample]

    for _ in range(1, n_train):
        # Calcula a menor distância de cada ponto ao conjunto já selecionado
        min_distances = np.min(distances[selected], axis=0)
        min_distances[selected] = -np.inf  # Evita escolher um ponto já selecionado
        next_sample = np.argmax(min_distances)  # Escolhe o mais distante

        selected.append(next_sample)

    return np.array(selected)


# Plot and save

In [12]:
def plot_data(outliers, T2, Q, T2_limit, Q_limit, cumulative_variance, optimal_components):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(cumulative_variance, label='Explained Variance Ratio')
    plt.xlabel('Number of Components')
    plt.ylabel('Explained Variance Ratio')
    plt.title('Explained Variance Ratio by Principal Components')
    plt.axvline(optimal_components - 1, color='r', linestyle='--', label=f'Optimal Components: {optimal_components}')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.scatter(T2, Q, c=outliers, cmap='coolwarm', edgecolor='k')
    plt.axhline(np.percentile(Q, 100 * (1 - 0.027)), color='r', linestyle='--', label=f'Q Limit: {np.percentile(Q, 100 * (1 - 0.02)):.2f}')
    plt.axvline(T2_limit, color='g', linestyle='--', label=f'T2 Limit: {T2_limit:.2f}')
    plt.xlabel("Hotelling's T²")
    plt.ylabel('F-residual')
    plt.title('Outlier Detection')
    plt.legend(title='Limits')

    plt.tight_layout()
    plt.show()

def save_datasets(calibration_set, validation_set, dataset_path, dataset_dir):
    base_name = os.path.splitext(os.path.basename(dataset_path))[0]  # Pega apenas o nome do arquivo sem extensão
    calibration_path = os.path.join(dataset_dir, f"{base_name}_calibracao.xlsx")
    validation_path = os.path.join(dataset_dir, f"{base_name}_validacao.xlsx")

    calibration_set.to_excel(calibration_path, index=False)
    validation_set.to_excel(validation_path, index=False)


In [13]:
def plot_interactive_scores(scores, identifiers, outliers):
    """
    Gera gráficos de scores interativos (PC1 vs PC2 e PC1 vs PC3) usando o Plotly.

    Args:
        scores (np.array): Array com os scores da PCA (amostras, componentes).
        identifiers (pd.DataFrame): DataFrame com as colunas de identificação das amostras.
        outliers (np.array): Array booleano indicando se uma amostra é outlier.
    """
    print("\nExibindo Gráficos de Scores Interativos (PC1 vs PC2 e PC1 vs PC3):")
    
    # Prepara um DataFrame para o Plotly
    pc_df = pd.DataFrame(scores, columns=[f'PC{i+1}' for i in range(scores.shape[1])])
    
    # Adiciona as informações de identificação e de outlier
    pc_df = pd.concat([identifiers.reset_index(drop=True), pc_df], axis=1)
    pc_df['Status'] = np.where(outliers, 'Outlier', 'Inlier') # 'Inlier' = não é outlier
    
    # ---- Gráfico PC1 vs PC2 ----
    fig1 = px.scatter(
        pc_df, 
        x='PC1', 
        y='PC2', 
        color='Status',  # Colore os pontos baseados no status de outlier
        color_discrete_map={'Outlier': 'red', 'Inlier': 'blue'}, # Define as cores
        hover_data=identifiers.columns, # Informação que aparece ao passar o mouse
        title='Gráfico de Scores: PC1 vs PC2'
    )
    fig1.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
    fig1.update_layout(xaxis_title="Score na Componente Principal 1", yaxis_title="Score na Componente Principal 2")
    fig1.show()

    # ---- Gráfico PC1 vs PC3 (se houver 3 ou mais componentes) ----
    if scores.shape[1] >= 3:
        fig2 = px.scatter(
            pc_df, 
            x='PC1', 
            y='PC3', 
            color='Status',
            color_discrete_map={'Outlier': 'red', 'Inlier': 'blue'},
            hover_data=identifiers.columns,
            title='Gráfico de Scores: PC1 vs PC3'
        )
        fig2.update_traces(marker=dict(size=8, opacity=0.8, line=dict(width=1, color='DarkSlateGrey')))
        fig2.update_layout(xaxis_title="Score na Componente Principal 1", yaxis_title="Score na Componente Principal 3")
        fig2.show()


In [14]:
def plot_interactive_scores_com_y(scores, identifiers, outliers, target_y_name, target_y_values):
    """
    Gera um gráfico de scores de PCA interativo usando Plotly.

    Args:
        scores (np.ndarray): Array com os scores do PCA (PC1, PC2, etc.).
        identifiers (pd.DataFrame): DataFrame com os metadados/identificadores.
        outliers (np.ndarray): Array booleano indicando quais amostras são outliers.
        target_y_name (str): O nome da variável alvo (ex: 'SST').
        target_y_values (pd.Series): Os valores da variável alvo para colorir os pontos.
    """
    # Cria um DataFrame para o plot
    df_plot = pd.DataFrame(scores[:, :2], columns=['PC1', 'PC2'])
    df_plot['ID'] = identifiers.iloc[:, 0].values # Pega a primeira coluna de identificadores
    df_plot['Outlier'] = ['Sim' if o else 'Não' for o in outliers]
    df_plot[target_y_name] = target_y_values.values

    fig = px.scatter(
        df_plot,
        x='PC1',
        y='PC2',
        color=target_y_name,  # Colore os pontos pelos valores da variável alvo
        symbol='Outlier',     # Usa símbolos diferentes para outliers
        hover_data=['ID', target_y_name], # Mostra o ID e o valor de Y ao passar o mouse
        title=f'Scores do PCA - Otimizado para {target_y_name}'
    )
    fig.show()

# Análise Exploratória

In [15]:
# Caminho base: sobe um nível a partir da pasta atual
base_dir = Path().resolve().parent

# Define os diretórios de interesse
dataset_origin = base_dir / "Data"
dataset_save_outliers = dataset_origin / "Original"
dataset_save_outliers.mkdir(parents=True, exist_ok=True) # Cria a pasta se não existir

In [18]:
# --- CONFIGURAÇÃO ---
# Caminho base: aponta para a pasta do projeto
base_dir = Path().resolve().parent

# Define os diretórios de interesse
dataset_origin_dir = base_dir / "Data" / "Original"

# Diretório para salvar os dados limpos e os outliers do Y
dataset_save_dir = base_dir / "Data"

# Lista de variáveis alvo
target_variables = ["SST", "PH", "UBS (%)", "FIRMEZA (N)", "AT"]

# Verifica se o diretório de origem existe
if not dataset_origin_dir.exists():
    raise FileNotFoundError(f"Pasta de origem não encontrada: {dataset_origin_dir}")

# --- LOOP PRINCIPAL ---
# Iterar sobre os arquivos na pasta de origem
for filename in os.listdir(dataset_origin_dir):
    if filename.endswith(".xlsx") and not filename.startswith('~'):
        print(f"--- Analisando arquivo: {filename} ---")
        dataset_path = dataset_origin_dir / filename
        df_original = pd.read_excel(dataset_path)

        # Guarda o nome base do arquivo
        base_name = os.path.splitext(filename)[0]
        
        # DataFrame para guardar todos os outliers encontrados
        all_outlier_indices = set()

        # 1. LOOP PARA IDENTIFICAR OUTLIERS EM CADA ATRIBUTO
        for attribute in target_variables:
            if attribute not in df_original.columns:
                print(f"Aviso: A coluna '{attribute}' não foi encontrada no arquivo {filename}. Pulando.")
                continue

            # --- Visualização com Box Plot Interativo ---
            fig = px.box(
                df_original, 
                y=attribute, 
                points="all", # Mostra todos os pontos
                hover_data=df_original.columns[:6], # Mostra os identificadores no hover
                title=f"Box Plot do Atributo: {attribute}"
            )
            fig.show()

            # --- Identificação Numérica via IQR ---
            Q1 = df_original[attribute].quantile(0.25)
            Q3 = df_original[attribute].quantile(0.75)
            IQR = Q3 - Q1

            # Define os limites para ser um outlier
            limite_inferior = Q1 - 1.5 * IQR
            limite_superior = Q3 + 1.5 * IQR

            # Encontra os outliers
            outliers_mask = (df_original[attribute] < limite_inferior) | (df_original[attribute] > limite_superior)
            outlier_indices = df_original[outliers_mask].index

            if not outlier_indices.empty:
                print(f"Outliers encontrados para '{attribute}': {len(outlier_indices)} amostra(s).")
                # Adiciona os índices encontrados ao nosso conjunto geral
                all_outlier_indices.update(outlier_indices)
            else:
                print(f"Nenhum outlier encontrado para '{attribute}'.")

        # 2. SEPARAR E SALVAR OS ARQUIVOS
        if all_outlier_indices:
            print(f"\nTotal de {len(all_outlier_indices)} amostra(s) únicas identificadas como outliers em pelo menos um atributo.")
            
            # Converte o conjunto de índices para uma lista para poder usar no .iloc
            outlier_indices_list = sorted(list(all_outlier_indices))

            # DataFrame contendo APENAS as linhas que são outliers em Y
            df_outliers_y = df_original.iloc[outlier_indices_list]
            
            # DataFrame contendo os dados LIMPOS (sem os outliers de Y)
            df_clean_y = df_original.drop(index=outlier_indices_list)

            # Salvar o arquivo com os outliers para documentação
            outlier_path = dataset_save_dir / f"{base_name}_Y_outliers.xlsx"
            df_outliers_y.to_excel(outlier_path, index=False)
            print(f"Arquivo com outliers de Y salvo em: {outlier_path}")

        else:
            print("\nNenhum outlier encontrado em nenhum atributo. O arquivo de dados limpos é igual ao original.")
            df_clean_y = df_original.copy() # Se não há outliers, o df limpo é uma cópia do original

        # Salvar o arquivo de dados limpos, que será usado na próxima etapa (PCA)
        clean_path = dataset_save_dir / f"{base_name}_Y_clean.xlsx"
        df_clean_y.to_excel(clean_path, index=False)
        print(f"Arquivo com dados limpos (sem outliers de Y) salvo em: {clean_path}\n")

--- Analisando arquivo: dataset_cotton_fruit.xlsx ---


Nenhum outlier encontrado para 'SST'.


Outliers encontrados para 'PH': 1 amostra(s).


Outliers encontrados para 'UBS (%)': 2 amostra(s).


Outliers encontrados para 'FIRMEZA (N)': 1 amostra(s).


Outliers encontrados para 'AT': 9 amostra(s).

Total de 11 amostra(s) únicas identificadas como outliers em pelo menos um atributo.
Arquivo com outliers de Y salvo em: C:\Users\luyza\Documents\ML-spectroscopy-analysis\Data\dataset_cotton_fruit_Y_outliers.xlsx
Arquivo com dados limpos (sem outliers de Y) salvo em: C:\Users\luyza\Documents\ML-spectroscopy-analysis\Data\dataset_cotton_fruit_Y_clean.xlsx



# Main

In [ ]:
base_dir = Path(r"C:\Users\luyza\Documents\ML-spectroscopy-analysis")
dataset_dir = base_dir / "Processed" / "snv_sg_osc_processed" # Pasta com os múltiplos arquivos do OSC
outliers_y_dir = base_dir / "Data"
dataset_save_pca = base_dir / "Processed" / "PCA"
dataset_save_outliers = dataset_save_pca / "Outliers_Identificados"

# Verifica se o diretório existe
if not dataset_dir.exists():
    raise FileNotFoundError(f"Pasta não encontrada: {dataset_dir}")

# Iterar sobre os arquivos na pasta
for filename in os.listdir(dataset_dir):
    if filename.endswith(".xlsx") and not filename.startswith('~'): # Ignora arquivos temporários do Excel
        print(f"--- Processando arquivo: {filename} ---")
        dataset_path = dataset_dir / filename
        df = pd.read_excel(dataset_path)

        try:
            target_y_name = filename.split('_for_')[1].split('.')[0]
            print(f"Variável alvo identificada: {target_y_name}")
        except IndexError:
            print(f"AVISO: Não foi possível identificar a variável alvo para o arquivo {filename}. Pulando este arquivo.")
            continue

        # Guarda o nome base do arquivo para usar depois
        base_name = os.path.splitext(filename)[0].replace("_processed", "")

        # montar o nome do arquivo de outliers do Y
        outlier_y_filename = f"dataset_cotton_fruit_Y_outliers.xlsx"
        outlier_y_path = outliers_y_dir / outlier_y_filename

        # verificar se o arquivo de outliers existe e removê-los
        if outlier_y_path.exists():
            print(f"Encontrado arquivo de outliers do Y: {outlier_y_filename}. Removendo amostras...")
            df_outliers_y = pd.read_excel(outlier_y_path)
            
            id_column_name = df.columns[0] 
            
            # Pega a lista de IDs dos outliers
            ids_to_remove = df_outliers_y[id_column_name].tolist()
            
            # Filtra o DataFrame principal 'df', mantendo apenas as linhas cujo ID NÃO ESTÁ na lista de remoção.
            initial_rows = len(df)
            df = df[~df[id_column_name].isin(ids_to_remove)]
            final_rows = len(df)
            
            print(f"{initial_rows - final_rows} amostra(s) removida(s) com base nos outliers do Y.")
            print(f"Tamanho do DataFrame após limpeza: {df.shape}")
        else:
            print(f"Aviso: Arquivo de outliers do Y '{outlier_y_filename}' não encontrado. Prosseguindo sem remover outliers do Y.")
        
        identifiers = df.iloc[:, :6]
        data = df.iloc[:, 6:].values

        # Isso será usado para colorir os gráficos
        if target_y_name in df.columns:
            y_target_values = df[target_y_name]
        else:
            print(f"AVISO: A coluna '{target_y_name}' não foi encontrada no DataFrame. Os gráficos não serão coloridos por ela.")
            # Cria uma série vazia para não quebrar o código
            y_target_values = pd.Series([None] * len(df), name=target_y_name)

        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data)
        
        # print("Shape dos dados normalizados:", data_scaled.shape)

        n_components = jackknife_pca(data_scaled)
        pca = PCA(n_components=n_components).fit(data_scaled)
        data_pca = pca.transform(data_scaled)

        # Para utilizar o Kennard_Stone antes de tirar os outlier é so descomentar
        # selected_indices = kennard_stone(data_pca)
        # print(f"Número de amostras na calibração: {len(selected_indices)}")  # Deve ser ~70% do total
        # calibration_set = df.iloc[selected_indices].sort_values(by=df.columns[0], ascending=True)
        # validation_set = df.drop(df.index[selected_indices])  # Em vez de usar a máscara negada
  
        T2 = np.sum((data_pca / np.std(data_pca, axis=0)) ** 2, axis=1)
        Q = np.sum((data_scaled - pca.inverse_transform(data_pca)) ** 2, axis=1)
        
        # Definição dos limites e identificação de outliers (seu código original)
        # Nota: Usar 3 desvios padrão é mais comum que 2 para pegar outliers mais extremos
        T2_limit = np.mean(T2) + 3 * np.std(T2)
        Q_limit = np.mean(Q) + 3 * np.std(Q)
        # Array booleano que define quem é outlier
        outliers = (T2 > T2_limit) | (Q > Q_limit)

        print(f"Número de outliers identificados por T²/Q: {np.sum(outliers)}")

        # Chamada para sua função de plot original
        cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
        #plot_data(outliers, T2, Q, T2_limit, Q_limit, cumulative_variance, n_components)

        plot_interactive_scores_com_y(data_pca, identifiers, outliers, target_y_name, y_target_values)
        
        # Chamar a nova função de plot interativo
        # Passamos os scores, os identificadores e a lista de outliers que já calculamos
        plot_interactive_scores(data_pca, identifiers, outliers)
        
        # Separação com Kennard-Stone e salvamento
        # (O ideal é remover os outliers ANTES de rodar o Kennard-Stone)
        df_outliers = df[outliers]
        df_clean = df[~outliers]
        if not df_outliers.empty:
            outlier_path = dataset_save_outliers / f"{base_name}_outliers.xlsx"
            df_outliers.to_excel(outlier_path, index=False)
            print(f"Arquivo com {len(df_outliers)} outliers salvo em: {outlier_path}")
        
        # selected_indices = kennard_stone(data_pca)
        # calibration_set = df.iloc[selected_indices].sort_values(by=df.columns[0], ascending=True)
        # validation_set = df.drop(df.index[selected_indices])
        # save_datasets(calibration_set, validation_set, dataset_path, dataset_save)
        # print(f"Arquivo {filename} processado com {n_components} componentes principais do PCA.\n")

        print(f"\nIniciando Kennard-Stone no conjunto de dados limpo de {len(df_clean)} amostras...")
        
        # Precisamos recalcular a PCA apenas com os dados limpos para o KS
        clean_data = df_clean.iloc[:, 6:].values
        clean_data_scaled = scaler.fit_transform(clean_data)
        
        # Usamos o mesmo número de componentes de antes para consistência
        pca_clean = PCA(n_components=n_components).fit(clean_data_scaled)
        clean_data_pca = pca_clean.transform(clean_data_scaled)
        
        # Roda o Kennard-Stone nos scores PCA dos dados LIMPOS
        selected_indices_clean = kennard_stone(clean_data_pca)
        
        # Os índices retornados pelo KS são baseados no df_clean, então usamos o .iloc
        calibration_set = df_clean.iloc[selected_indices_clean].sort_values(by=df.columns[0], ascending=True)
        validation_set = df_clean.drop(df_clean.index[selected_indices_clean])
        
        print(f"Número de amostras na calibração (limpa): {len(calibration_set)}")
        print(f"Número de amostras na validação (limpa): {len(validation_set)}")
        
        # Salva os novos datasets de calibração e validação (sem outliers)
        # Usamos a função save_datasets original, mas passamos os novos dataframes
        save_datasets(calibration_set, validation_set, dataset_path, dataset_save_pca)

        print(f"Arquivo {filename} processado. Datasets de calibração/validação limpos salvos na pasta PCA.\n")



PROCESSANDO PASTA DE FILTRO: continuum_removal

--- Processando arquivo: continuum_removal.xlsx ---
Nenhuma variável alvo específica encontrada. Processando como filtro independente.
Resultados serão salvos em: C:\Users\luyza\Documents\ML-spectroscopy-analysis\PCA_Results\continuum_removal\continuum_removal
Removendo amostras com base no arquivo: dataset_cotton_fruit_Y_outliers.xlsx.
0 amostra(s) removida(s). DataFrame agora com 239 amostras.
Número de outliers de PCA (T²/Q) identificados: 9
Arquivo com 9 outliers de PCA salvo em: C:\Users\luyza\Documents\ML-spectroscopy-analysis\PCA_Results\continuum_removal\continuum_removal\Outliers_Identificados\continuum_removal_PCA_outliers.xlsx

Iniciando Kennard-Stone no conjunto de dados limpo de 230 amostras...
Número de amostras na calibração (limpa): 161
Número de amostras na validação (limpa): 69
Arquivo continuum_removal.xlsx processado com sucesso.


PROCESSANDO PASTA DE FILTRO: emsc

--- Processando arquivo: emsc.xlsx ---
Nenhuma variá

KeyboardInterrupt: 

# Avaliação dos filtros depois do PCA

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import warnings

# Ignorar avisos que podem aparecer durante a convergência do PLS
warnings.filterwarnings('ignore', category=UserWarning)

# =============================================================================
# --- CONFIGURAÇÃO DA ANÁLISE ---
# =============================================================================

# 1. Defina o caminho base do seu projeto
base_dir = Path(r"C:\Users\luyza\Documents\ML-spectroscopy-analysis")

# 2. Defina a variável-alvo que você quer usar para esta avaliação rápida.
#    Escolha a mais importante para o seu projeto.
PRIMARY_TARGET_VARIABLE = "SST"

# 3. Defina o número máximo de componentes do PLS que você quer testar.
#    Um valor entre 15 e 25 é geralmente suficiente.
MAX_PLS_COMPONENTS = 15

# =============================================================================
# --- INÍCIO DO SCRIPT DE AVALIAÇÃO ---
# =============================================================================

# Diretório raiz que contém todos os resultados do PCA
pca_results_root_dir = base_dir / "PCA_Results"

# Lista para armazenar os resultados finais de cada filtro
all_filter_results = []

print("="*60)
print(f"INICIANDO AVALIAÇÃO DE FILTROS PARA A VARIÁVEL: '{PRIMARY_TARGET_VARIABLE}'")
print("="*60)

# Verificação inicial
if not pca_results_root_dir.exists():
    raise FileNotFoundError(f"Diretório de resultados do PCA não encontrado: {pca_results_root_dir}")

# Sanitiza o nome da variável alvo para comparar com os nomes das pastas
sanitized_target = PRIMARY_TARGET_VARIABLE.replace(' (%)', '').replace(' (N)', '').replace(' ', '_')

# Loop 1: Itera sobre as pastas de cada tipo de filtro (ex: 'msc', 'snv_sg_osc')
for filter_folder_name in os.listdir(pca_results_root_dir):
    filter_folder_path = pca_results_root_dir / filter_folder_name
    if not filter_folder_path.is_dir():
        continue

    # Loop 2: Itera sobre a pasta de cada resultado específico (ex: 'msc', 'snv_sg_osc_SST')
    for result_folder_name in os.listdir(filter_folder_path):
        result_folder_path = filter_folder_path / result_folder_name
        
        # Define o nome do filtro que será usado no relatório final
        # Ex: "snv_sg_osc (para SST)" ou "msc (independente)"
        report_name = f"{filter_folder_name}/{result_folder_name}"
        
        # Condição para processar:
        # 1. Se o nome da pasta termina com o alvo sanitizado (ex: '_SST')
        # 2. Ou se o nome da pasta é igual ao nome do filtro (caso de filtros independentes como 'msc')
        is_target_dependent_match = result_folder_name.endswith(f"_{sanitized_target}")
        is_independent_filter = result_folder_name == filter_folder_name

        if not (is_target_dependent_match or is_independent_filter):
            continue # Pula para o próximo se não for o alvo que queremos analisar agora

        # Constrói o caminho para o arquivo de calibração
        cal_filename = f"{result_folder_name}_cal.xlsx"
        cal_filepath = result_folder_path / cal_filename

        if not cal_filepath.exists():
            continue

        print(f"\n--- Analisando filtro: {report_name} ---")

        # Carrega os dados de calibração
        df_cal = pd.read_excel(cal_filepath)

        # Verifica se a coluna alvo existe no arquivo
        if PRIMARY_TARGET_VARIABLE not in df_cal.columns:
            print(f"  Aviso: Coluna '{PRIMARY_TARGET_VARIABLE}' não encontrada. Pulando.")
            continue
        
        # Separa os dados X e a variável alvo y
        X_cal = df_cal.iloc[:, 6:] # Assume que os dados espectrais começam na 7ª coluna
        y_cal = df_cal[PRIMARY_TARGET_VARIABLE]
        
        # Garante que não há valores nulos que possam quebrar a análise
        if y_cal.isnull().any():
            print(f"  Aviso: Valores nulos encontrados em '{PRIMARY_TARGET_VARIABLE}'. Removendo linhas correspondentes.")
            not_null_mask = y_cal.notnull()
            X_cal = X_cal[not_null_mask]
            y_cal = y_cal[not_null_mask]

        # Normaliza os dados (importante para PLS)
        scaler = StandardScaler()
        X_cal_scaled = scaler.fit_transform(X_cal)

        # --- Loop para encontrar o melhor número de componentes para este filtro ---
        best_rmse = float('inf')
        best_n_components = 0
        
        print(f"  Testando PLS com 1 a {MAX_PLS_COMPONENTS} componentes...")
        
        for n in range(1, MAX_PLS_COMPONENTS + 1):
            # Cria o modelo PLS
            pls = PLSRegression(n_components=n)
            
            # Executa a validação cruzada (cv=5)
            # Usamos 'neg_root_mean_squared_error', então o resultado será negativo.
            scores = cross_val_score(
                pls, X_cal_scaled, y_cal, 
                cv=5, scoring='neg_root_mean_squared_error'
            )
            
            # Converte o score para um RMSE positivo
            current_rmse = -np.mean(scores)
            
            # Verifica se este é o melhor resultado até agora
            if current_rmse < best_rmse:
                best_rmse = current_rmse
                best_n_components = n
        
        print(f"  -> Melhor Resultado: RMSE = {best_rmse:.4f} com {best_n_components} componentes.")
        
        # Guarda o melhor resultado para o ranking final
        all_filter_results.append({
            "Filtro": report_name,
            "Melhor N Componentes": best_n_components,
            "RMSE (Validação Cruzada)": best_rmse
        })

# --- GERAÇÃO DO RELATÓRIO FINAL ---
print("\n\n" + "="*60)
print(f"RANKING FINAL DOS FILTROS (Menor RMSE é Melhor)")
print("="*60)

if not all_filter_results:
    print("Nenhum resultado foi gerado. Verifique os caminhos e o nome da variável alvo.")
else:
    # Cria um DataFrame com os resultados
    results_df = pd.DataFrame(all_filter_results)
    
    # Ordena o DataFrame pelo RMSE, do menor para o maior
    results_df_sorted = results_df.sort_values(by="RMSE (Validação Cruzada)", ascending=True)
    
    # Imprime o ranking
    print(results_df_sorted.to_string(index=False))
    
    # Anuncia o vencedor
    best_filter = results_df_sorted.iloc[0]
    print("\n🏆 Filtro mais promissor:", best_filter['Filtro'])
    print(f"   Com um RMSE de {best_filter['RMSE (Validação Cruzada)']:.4f} usando {best_filter['Melhor N Componentes']} componentes.")

print("\n✅ Avaliação concluída!")